In [27]:
import joblib
import numpy as np
import xgboost as xgb
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

ModuleNotFoundError: No module named 'xgboost'

In [26]:
def test01():

    # 1. 加载训练数据
    data = pd.read_csv('data/红酒品质分类.csv')
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1] - 3  # 原始数据是从3开始的类别, 这里给它改成从0开始

    # 2. 数据集分割
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, stratify=y, random_state=22)

    # 3. 存储数据
    pd.concat([x_train, y_train], axis=1).to_csv('data/红酒品质分类-train.csv')
    pd.concat([x_valid, y_valid], axis=1).to_csv('data/红酒品质分类-valid.csv')

In [3]:
test01()

In [4]:
data = pd.read_csv('data/红酒品质分类.csv')

In [6]:
data['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [15]:
train_data = pd.read_csv('data/红酒品质分类-train.csv')
valid_data = pd.read_csv('data/红酒品质分类-valid.csv')

x_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1] # 最后一列作为目标值

x_valid = valid_data.iloc[:,:-1]
y_valid = valid_data.iloc[:,-1]

estimator = xgb.XGBClassifier(n_estimators = 100,
                              objective = 'multi:softmax',
                              eval_metric = 'merror',
                              eta = 0.1,use_label_encoder = False,random_state = 22)
estimator.fit(x_train,y_train)
y_pred = estimator.predict(x_valid)
from sklearn.utils import class_weight
class_weights = class_weight.compute_sample_weight(class_weight='balanced',y=y_train)
estimator.fit(x_train,y_train,sample_weight = class_weights)
y_pred = estimator.predict(x_valid)

In [16]:
print(classification_report(y_valid,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.29      0.18      0.22        11
           2       0.72      0.84      0.77       136
           3       0.69      0.58      0.63       128
           4       0.55      0.57      0.56        40
           5       0.20      0.33      0.25         3

    accuracy                           0.67       320
   macro avg       0.41      0.42      0.41       320
weighted avg       0.66      0.67      0.66       320


In [19]:
from sklearn.model_selection import GridSearchCV
# 交叉验证网格搜索
# 交叉验证，网格搜索
train_data = pd.read_csv('data/红酒品质分类-train.csv')
valid_data = pd.read_csv('data/红酒品质分类-valid.csv')

# 训练集
x_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]

# 测试集
x_valid = valid_data.iloc[:, :-1]
y_valid = valid_data.iloc[:, -1]

spliter = StratifiedKFold(n_splits=5,shuffle=True,random_state=10) # 跟GridSearchCV配合使用 比直接cv=5效果要更好
param_grid = {'max_depth': np.arange(3, 5, 1),
              'n_estimators': np.arange(50, 150, 50),
              'eta': np.arange(0.1, 1, 0.3)}
estimator = xgb.XGBClassifier(n_estimators=100,  # 学习器数量
                              objective='multi:softmax', #目标函数 多分类 softmax
                              eval_metric='merror', # 多分类错误率
                              eta=0.1,  #实际上就是learnign rate alpha正则项
                              use_label_encoder=False, #是不是顺序编码  label encoder  one-hot encoder
                              random_state=22)
cv = GridSearchCV(estimator,param_grid,cv=spliter)
cv.fit(x_train,y_train)
y_pred = cv.predict(x_valid)
print(classification_report(y_true=y_valid, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00        11
           2       0.69      0.83      0.76       136
           3       0.65      0.58      0.61       128
           4       0.60      0.60      0.60        40
           5       0.50      0.33      0.40         3

    accuracy                           0.66       320
   macro avg       0.41      0.39      0.39       320
weighted avg       0.64      0.66      0.65       320


/root/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
cv.best_params_

{'eta': 0.7000000000000001, 'max_depth': 4, 'n_estimators': 100}